<h1 style=" text-align:center; color: blueviolet; margin-top:3%" > Web scraping con Python y bs4 </h1>



<p style=" margin-top:5%;width:80%; margin-left:auto; margin-right:auto;padding:2% ">
    La idea del trabajo es obtener información sobre los casos de Covid-19 en America Latina 
    desde la página Wikipedia. <br>
    <br>
     <i> ( url: https://en.wikipedia.org/w/index.php?title=Template:COVID-19_testing_by_country ). </i> <br>
    <br>
    En dicha página habrá una tabla con datos de Covid-19 de varios paises. El trabajo consiste en capturar esos     
    datos y filtrar según los paises en cuestión. <br>
    
   

</p>



In [1]:
# request py - Para peticiones http
import requests

# beatifulSoup para web Scraping
from bs4 import BeautifulSoup

# pandas, para crear el dataFrame
import pandas as pd




In [2]:
# Se guarda la página objetivo, (en este caso wiki_covid) con requests

url = "https://en.wikipedia.org/w/index.php"

wiki_covid = requests.get(url, params = { "title": "Template:COVID-19_testing_by_country" })

# print(wiki_covid.text)

In [3]:
# se utiliza BeatifulSoup() para poder despues navegar sobre la pagina 

wikiCovid = BeautifulSoup(wiki_covid.text,"html.parser")    # se crea el objeto BeatifulSoup

# En la página wikiCovid, nos interesa la tabla, class="wikitable" ... 
wikiTable = wikiCovid.find_all("table",class_="wikitable")  # Buscamos y guardamos la tabla para analizar

# print( wikiTable[0].prettify() ) 



In [4]:
# Obtenemos las filas de la tabla (de wikiTable) 
filas = wikiTable[0].find_all("tr") 

#
# Trabajamos con Pandas para crear el dataframe 
#

v = []

for j in range(len(filas)):
    prueba = filas[j].get_text().strip().split("\n")
    v.append([prueba[2*i] for i in range(len(prueba)//2)] ) 
    
dataSet = {
    "Pais" : [v[i][0] for i in range(1,len(v)) ],
    "Casos_Confirmados" : [v[i][4] for i in range(1,len(v)) ],
    "Fecha" : [v[i][1] for i in range(1,len(v)) ]
}

dataFrame = pd.DataFrame(data=dataSet)

dataFrame.head(10)



,Pais,Casos_Confirmados,Fecha
0,Afghanistan,"49,621",17 Dec 2020
1,Albania,"96,838",18 Feb 2021
2,Algeria,"58,574",2 Nov 2020
3,Andorra,"37,958",23 Feb 2022
4,Angola,"20,981",2 Feb 2021
5,Antigua and Barbuda,832,6 Mar 2021
6,Argentina,"9,060,495",16 Apr 2022
7,Armenia,"422,963",29 May 2022
8,Australia,"10,112,229",9 Sep 2022
9,Austria,"5,789,991",1 Feb 2023


In [5]:

# arreglo de latinos 
latinos = ["Uruguay","Argentina","Chile","Brazil","Paraguay","Bolivia","Peru","Ecuador","Colombia","Venezuela"]

# dataFrame de latinos

for j in range(len(list(dataFrame.columns))):
    datosLatinos = pd.DataFrame( { dataFrame.columns[j] : [] } )

datosLatinos

for i in range(len(latinos)):
    datosLatinos = pd.concat([datosLatinos, dataFrame.loc[ dataFrame["Pais"] == latinos[i] ]] ) 

datosLatinos


,Fecha,Pais,Casos_Confirmados
166,16 Apr 2022,Uruguay,"895,592"
6,16 Apr 2022,Argentina,"9,060,495"
32,1 Feb 2023,Chile,"5,123,007"
23,19 Feb 2021,Brazil,"10,081,676"
127,27 Mar 2022,Paraguay,"647,950"
20,5 Jun 2022,Bolivia,"910,228"
128,17 Nov 2022,Peru,"4,177,786"
45,23 Jul 2021,Ecuador,"480,720"
34,24 Nov 2022,Colombia,"6,314,769"
168,30 Mar 2021,Venezuela,"159,149"


In [71]:
# Exportamos la tabla datosLatinos, ... covidLatino.csv

datosLatinos.to_csv("covidLatino.csv")

## Se guarda en la misma ubicación que este archivo notebook. 
